<a href="https://colab.research.google.com/github/mmonch/Sidecar_Project/blob/main/notebooks/Sidecar_Project_pre_processing_and_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The aim is to pre-process and standardized the attribute business names in oder to have an easier time mapping the tech names to the business names.

The idea is to leverage fasttext to encode the business attributes and find synonyms in the data set.

the find and replace to standardize and minimize the amount of categories in the business names.

IDEA: sentence transforming d2 --> library --> get embeddings --> similarity

Fasttext can also be used to get mappings for out of vocabulary (oov) words that do not appear in the training set.

how fasttext works:
https://towardsdatascience.com/fasttext-under-the-hood-11efc57b2b3

how to use pre-trained fasttext models:
https://stackoverflow.com/questions/67091670/loading-pre-trained-fasttext-model
https://fasttext.cc/docs/en/supervised-models.html


# small subset --> show prelimiary results
dictionary by:
- lookup
- similarity matching
etc.



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Supervised Models
use --> 
 -qnorm -retrain -cutoff 100000 
to further train model

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 5.3 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3133039 sha256=45485c67ac8c4e401c9e79a71cc955ec30b333af7052df255026c73e07534684
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
# load  pre-trained model
from gensim.models import FastText


#from FastText import *

data_path = "gdrive/My Drive/SIDECAR_P/dbpedia.bin"
model = fasttext.load_model(data_path)

AttributeError: ignored

In [ ]:

# command to further train pre-trained model
!-qnorm -retrain -cutoff 100000

In [ ]:
# install modules/packages
!pip install contractions

     |████████████████████████████████| 284 kB 6.8 MB/s 
     |████████████████████████████████| 321 kB 90.5 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85450 sha256=8f9713c6319ed7e4aa3285c57122179265c9cbda11a1dd4c76b647069656c444
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
# General
import pandas as pd
import numpy as np
# progress bar
import tqdm
# NLP specific
import unicodedata
import re
import contractions

In [ ]:
data = pd.read_csv("gdrive/My Drive/SIDECAR_P/Sidecar_Data_Sample.csv")

In [ ]:
top_n= data.Attribute_Business_Name.value_counts()[:100]
top_n_df = data[data['Attribute_Business_Name'].map(data['Attribute_Business_Name'].value_counts()).isin(top_n)]


In [ ]:
# preprocess and normalize Text

# in case text not english
def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

# preprocessing
def pre_process_text(labels):
  norm_docs = []
  for string in tqdm.tqdm(labels):
    string = string.replace("_", " ")
    string = string.translate(string.maketrans("\n\t\r", "   "))
    string = remove_accented_chars(string) 
    # and inset a space where a number follows a letter et vice versa
    string = re.sub(r'(?<=\d)(?=[^\d\s])|(?<=[^\d\s])(?=\d)', ' ', string)
    # insert space where an uppercase letter follows a lowercase letter
    string = re.sub(r"(?<![A-Z\W])(?=[A-Z])", " ", string)
    string = contractions.fix(string)
    # where XXXX number to XXXX number replace - with to IN PROGRESS
    string = string.replace("-", " to ")
    # remove special characters or whitespaces
    string = re.sub(r"[^a-zA-Z0-9\s]", "", string, flags=re.I|re.A)
    string = string.lower()
    string = string.strip()
    # optional: split 
    for word in string.split(" "):
      norm_docs.append(word)
  return norm_docs

In [ ]:
# remove P_AF18XXXXX strings for better model performance
#data_no_paf = data[data['Attribute_Technical_Name'].str.contains("P_AF")==False]

In [ ]:
#data_no_paf

In [ ]:
tech_label_list = pre_process_text(data["Attribute_Technical_Name"])


In [ ]:
business_label_list = pre_process_text(data["Attribute_Business_Name"])


In [ ]:
all_words = tech_label_list + business_label_list

In [ ]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model = FastText()

# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

print(model)

ModuleNotFoundError: ignored

In [ ]:
# fasttext
!pip install fasttext


In [ ]:
import fasttext
from fasttext import *

In [ ]:
help(fasttext.FastText)

mapping --> start with frequent

edit distance, similarity --> dict

In [ ]:
embedding_file_path = "gdrive/My Drive/SIDECAR_P/wiki-news-300d-1M.vec"
def get_coefs(word,*arr):
        return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*row.split(" "))
for row in open(embedding_file_path, encoding="utf8", errors='ignore')
if len(row)>100)

In [ ]:
list(embeddings_index.items())[247:248]

In [ ]:
list(embeddings_index.keys())[:100]

In [ ]:
from scipy.spatial import distance
result1 = 1 - distance.cosine(embeddings_index["children"], embeddings_index["table"])
result1

0.3573344945907593

In [ ]:
result2 = 1 - distance.cosine(embeddings_index["change"], embeddings_index["update"])
result2

0.6034049391746521

https://towardsdatascience.com/understanding-nlp-word-embeddings-text-vectorization-1a23744f7223
https://en.wikipedia.org/wiki/Cosine_similarity

Word similarity aka Cosine similarity of vectors A and B is
sum i through n of Ai * Bi
divided by
squareroot of sum i though n of Ai^2 * squareroot of sum i though n of Bi^2

with our newfound cosine similarity powers, we can:
- preprocess technical name inputs
- remove stopwords from businessinputs
- look business inputs up in the embeddings_index (or different versions)
- calculate distances between business input words 
- search and replace very similar words (do we need a cut-off value)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
embeddings_index

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("the")
data_no_paf["business_no_stopwords"] = data_no_paf["prep_business_data"].apply(lambda x: " ".join([word for word in x.split() if word not in (stopwords)]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# look up business_no_stopwords in embeddings_index
data_no_paf["business_no_stopwords"]

0                technical id patient
1                         gender code
2                  living arrangement
3                 marital status code
4                     occupation code
                    ...              
4128    config asset list data source
4129            attribute sample data
4130                      property dq
4131          property dq calculation
4132        property dq output format
Name: business_no_stopwords, Length: 3941, dtype: object

In [ ]:
def find_embeddings(word):
  if word in list(embeddings_index.keys()):
    return embeddings_index[word]
  else:
    return "SNAFU!"

In [ ]:
business_words_df = data_no_paf.business_no_stopwords.str.split(expand=True)

business_words_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,technical,id,patient,None,None,None,None,None,None,None,None,None,None,None,None
1,gender,code,None,None,None,None,None,None,None,None,None,None,None,None,None
2,living,arrangement,None,None,None,None,None,None,None,None,None,None,None,None,None
3,marital,status,code,None,None,None,None,None,None,None,None,None,None,None,None
4,occupation,code,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,config,asset,list,data,source,None,None,None,None,None,None,None,None,None,None
4129,attribute,sample,data,None,None,None,None,None,None,None,None,None,None,None,None
4130,property,dq,None,None,None,None,None,None,None,None,None,None,None,None,None
4131,property,dq,calculation,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
# replace all None with NaN

business_words_df = business_words_df.fillna(value=np.nan)


In [ ]:
business_words_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,technical,id,patient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gender,code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,living,arrangement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,marital,status,code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,occupation,code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128,config,asset,list,data,source,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4129,attribute,sample,data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4130,property,dq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4131,property,dq,calculation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!pip install tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
business_words_embeddings_df = business_words_df.applymap(find_embeddings)


KeyboardInterrupt: ignored

In [ ]:
business_words_embeddings_df

NameError: ignored

In [ ]:
data_no_paf.loc[data_no_paf["business_embeddings"] == "SNAFU!"].count()

In [ ]:
from gensim.models import FastText
from gensim.test.utils import common_texts

In [ ]:
print(common_texts[0])
print(len(common_texts))

['human', 'interface', 'computer']
9


In [ ]:
model = FastText(vector_size=4, window=3, min_count=1)
model.build_vocab(corpus_iterable=common_texts) # corpus_iterable needs to be a list of words
model.train(corpus_iterable=common_texts, total_examples=len(common_texts), epochs=10)

(36, 290)

## gensim.models.fasttext - FastText model

Source: https://radimrehurek.com/gensim/models/fasttext.html


In [ ]:
# fasttext from gen sim
from gensim.models import FastText
model_ted = FastText(sentences_ted, size=100, window=5, min_count=5, workers=4,sg=1)

NameError: ignored

Training hyperparameters
Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec:

- model: Training architecture. Allowed values: cbow, skipgram (Default cbow)

- vector_size: Dimensionality of vector embeddings to be learnt (Default 100)

- alpha: Initial learning rate (Default 0.025)

- window: Context window size (Default 5)

- min_count: Ignore words with number of occurrences below this (Default 5)

- loss: Training objective. Allowed values: ns, hs, softmax (Default ns)

- sample: Threshold for downsampling higher-frequency words (Default 0.001)

- negative: Number of negative words to sample, for ns (Default 5)

- epochs: Number of epochs (Default 5)

- sorted_vocab: Sort vocab by descending frequency (Default 1)

- threads: Number of threads to use (Default 12)

In addition, fastText has three additional parameters:

- min_n: min length of char ngrams (Default 3)

- max_n: max length of char ngrams (Default 6)

- bucket: number of buckets used for hashing ngrams (Default 2000000)

Parameters min_n and max_n control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If max_n is set to 0, or to be lesser than min_n, no character ngrams are used, and the model effectively reduces to Word2Vec.

To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the Fowler-Noll-Vo hashing function (FNV-1a variant) is employed.

Note: You can continue to train your model while using Gensim’s native implementation of fastText.



In [ ]:
#import zipfile
#from google.colab import drive

#drive.mount('/content/drive/')

#!ls "/content/drive/My Drive/ SIDECAR_P"
#!unzip -q "/content/drive/My Drive/SIDECAR_P/wiki.en_2/wiki.en.zip"
#zip_ref = zipfile.ZipFile("/content/drive/My Drive/SIDECAR_P/wiki.en.zip", 'r')
#zip_ref.extractall("/tmp")
#zip_ref.close()


Mounted at /content/drive/
ls: cannot access '/content/drive/My Drive/ SIDECAR_P': No such file or directory
unzip:  cannot find or open /content/drive/My Drive/SIDECAR_P/wiki.en_2/wiki.en.zip, /content/drive/My Drive/SIDECAR_P/wiki.en_2/wiki.en.zip.zip or /content/drive/My Drive/SIDECAR_P/wiki.en_2/wiki.en.zip.ZIP.


In [ ]:
!pip install gensim --upgrade
import gensim
print(gensim.__version__)
# new_sentences = 
# model.build_vocab(new_sentences, update=True)

     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
4.1.2


In [ ]:
# load  pre-trained model
from gensim.models import FastText


#from FastText import *

data_path = "gdrive/My Drive/SIDECAR_P/wiki.en_2/wiki.en.bin"
model = FastText.load(data_path)

UnpicklingError: ignored

In [ ]:
from gensim.models.fasttext import load_facebook_model
model = load_facebook_model(data_path)
# session crahsed after using all available RAM

In [ ]:
from gensim.models import FastText
data_path = "gdrive/My Drive/SIDECAR_P/wiki.en_2/wiki.en.bin"
model = FastText.load_fasttext_format(data_path) 
# session crahsed after using all available RAM

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model = FastText(vector_size=100)

# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

print(model)

In [ ]:
# load  pre-trained model
cap_path = datapath("crime-and-punishment.bin")
fb_model = load_facebook_model(cap_path)

'''Be sure to call the build_vocab() method with update=True before the train() 
method when continuing training. Without this call, previously unseen terms 
will not be added to the vocabulary.'''

'computer' in fb_model.wv.key_to_index  # New word, currently out of vocab
# False
old_computer = np.copy(fb_model.wv['computer'])  # Calculate current vectors
# include this line to continue training
fb_model.build_vocab(new_sentences, update=True)
fb_model.train(new_sentences, total_examples=len(new_sentences), epochs=model.epochs)
new_computer = fb_model.wv['computer']
np.allclose(old_computer, new_computer, atol=1e-4)  # Vector has changed, model has learnt something
# False
'computer' in fb_model.wv.key_to_index  # New word is now in the vocabulary
# True

NameError: ignored

In [ ]:
# Retrieve the word-vector for vocab and out-of-vocab word:

existent_word = "computer"
existent_word in model.wv.key_to_index
# True
computer_vec = model.wv[existent_word]  # numpy vector of a word
oov_word = "graph-out-of-vocab"
oov_word in model.wv.key_to_index
# False
oov_vec = model.wv[oov_word]  # numpy vector for OOV word

In [ ]:
# You can perform various NLP word tasks with the model, some of them are already built-in:
similarities = model.wv.most_similar(positive=['computer', 'human'], negative=['interface'])
most_similar = similarities[0]

similarities = model.wv.most_similar_cosmul(positive=['computer', 'human'], negative=['interface'])
most_similar = similarities[0]

not_matching = model.wv.doesnt_match("human computer interface tree".split())

sim_score = model.wv.similarity('computer', 'human')

In [ ]:
print(wv.most_similar("nights"))
print(wv.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant']))
print(wv.most_similar(positive=['baghdad', 'england'], negative=['london']))

In [ ]:
# Correlation with human opinion on word similarity:
from gensim.test.utils import datapath

similarities = model.wv.evaluate_word_pairs(datapath('wordsim353.tsv'))

In [ ]:
#Once you have a model, you can access its keyed vectors via the model.wv attributes. The keyed vectors instance is quite powerful: it can perform a wide range of NLP tasks. For a full list of examples, see KeyedVectors.
model_ted.wv.most_similar("Gastroenteritis")

## Fasttext Documentation

Source: https://fasttext.cc/docs/en/supervised-models.html

In [ ]:
!pip install fasttext
# https://fasttext.cc/docs/en/supervised-models.html
# pre-trained model --> retrain
-qnorm -retrain -cutoff 100000

In [ ]:
# load pre-trained .bin model that can provide embeddings for unseen words OOV --> USEFUL for trad. mapping approach
fasttext.load_model('file.bin')
# what is .bin? it is a  BINARY FASTTEXT MODEL --> can be trained more

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 4.6 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3128288 sha256=93f771f1220a26d4c705b6e073b2309e186f381f05c5d05d795981b783d14a0a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext
file_path = "/gdrive/My Drive/SIDECAR_P/wiki.en.bin"
model1 = fasttext.load_model(file_path)

ValueError: ignored

Yet another version of Fasttext

In [ ]:
!pip install fasttext-win

In [ ]:
import fasttext

model = fasttext.load_model('/gdrive/My Drive/SIDECAR_P/wiki.en/wiki.en.bin')

ValueError: ignored